<a href="https://www.arduino.cc/"><img src="https://raw.githubusercontent.com/sandeepmistry/aimldevfest-workshop-2019/master/images/Arduino_logo_R_highquality.png" width=200/></a>
# Tiny ML on Arduino
## Gesture recognition tutorial
## Original Code from:
 * Sandeep Mistry - Arduino
 * Don Coleman - Chariot Solutions

https://github.com/arduino/ArduinoTensorFlowLiteTutorials/

## Modified by:
 * Kusuma Wardana



## Seting Lingkungan Python

Install beberapa pustaka yang diperlukan. <b>Tensorflow</b> yang digunakan yang digunakan adalah versi 2. <b>Numpy</b> digunakan untuk operasi/pengolahan matriks dan <b>Matplotlib</b> adalah pustaka untuk visualisasi data. Pustaka <b>xxd</b> digunakan untuk mengubah model yang dihasilkan oleh Tensorflow menjadi bentuk matriks heksadesimal yang nantinya akan dipakai dalam pemrograman Arduino

In [ ]:
# Setup environment
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow==2.0.0-rc1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Unggah Data yang Diperlukan

1. Klik lambang folder di sisi kiri
2. Pilih Upload to Session Storage
3. Cari dan pilih 'telungkup.csv' dan 'tengadah.csv'

# Visualisasi Data

Data terdiri dari dua jenis input, yaitu data Accelerator and Gyroscope. Masing-masing memiliki Unit dan skala nilai yang berbeda

### Ploting telungkup.csv

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

filename = "telungkup.csv"

df = pd.read_csv("/content/" + filename)

index = range(1, len(df['aX']) + 1)

plt.rcParams["figure.figsize"] = (20,10)

plt.plot(index, df['aX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['aY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['aZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Acceleration")
plt.xlabel("Sample #")
plt.ylabel("Acceleration (G)")
plt.legend()
plt.show()

plt.plot(index, df['gX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['gY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['gZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Gyroscope")
plt.xlabel("Sample #")
plt.ylabel("Gyroscope (deg/sec)")
plt.legend()
plt.show()


### Ploting tengadah.csv

In [ ]:
filename = "tengadah.csv"

df = pd.read_csv("/content/" + filename)

index = range(1, len(df['aX']) + 1)

plt.rcParams["figure.figsize"] = (20,10)

plt.plot(index, df['aX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['aY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['aZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Acceleration")
plt.xlabel("Sample #")
plt.ylabel("Acceleration (G)")
plt.legend()
plt.show()

plt.plot(index, df['gX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['gY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['gZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Gyroscope")
plt.xlabel("Sample #")
plt.ylabel("Gyroscope (deg/sec)")
plt.legend()
plt.show()

# Latih Model Neural Network





## Parsing Data dan Lakukan Preprocessing

Parsing file csv dan lakukan normalisasi data. Proses normalisasi dalam percobaan ini akan mengubah nilai input menjadi rentang antara 0-1. Model yang akan digunakan adalah Fully Connected Layer (Multilayer Perceptron).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print(f"Versi TensorFlow = {tf.__version__}\n")

# Set 'seed' untuk Tensorflow dan Numpy, sehingga data random yang dibangkitkan
# sama, sehingga setiap orang yang menjalankan program ini akan menghasilkan
# output yang sama. 

SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Daftar gestur yang terlibat. Gunakan nama yang sama dengan nama file
GESTURES = [
    "telungkup",
    "tengadah",
]

SAMPLES_PER_GESTURE = 119  #frekuensi sampling

NUM_GESTURES = len(GESTURES)  # saat ini = 2

# Buat matriks'one-hot' enkoding untuk keperluan klasifikasi
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# baca kedua csv file, dan buat input and output untuk model
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(f"Processing index {gesture_index} untuk gesture '{gesture}'.")
  
  output = ONE_HOT_ENCODED_GESTURES[gesture_index]
  
  df = pd.read_csv("/content/" + gesture + ".csv")
  
  # Jumlah input adalah total data dibagi 119 = 100 
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)
  
  print(f"\tTerdapat {num_recordings} data terekam untuk {gesture}.")
  
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      # normalisasi input menjadi rentang 0 - 1
      # - Nilai accelerator antara: -4 to +4
      # - Nilai gyroscope antara: -2000 to +2000
      tensor += [
          (df['aX'][index] + 4) / 8,
          (df['aY'][index] + 4) / 8,
          (df['aZ'][index] + 4) / 8,
          (df['gX'][index] + 2000) / 4000,
          (df['gY'][index] + 2000) / 4000,
          (df['gZ'][index] + 2000) / 4000
      ]

    inputs.append(tensor)
    outputs.append(output)

# Konversi ke numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)

print("Persiapan dan Parsing Data Selesai!")

## Acak data dan bagi pasangan input-output untuk dilatih

Secara random, bagi pasangan data input-output sebagai berikut:
  - 60% untuk data latih (training data)
  - 20% untuk data validasi (validation data)
  - 20% untuk data uji (testing data)

**Training data** digunakan untuk melatih model. **Validation data** digunakan untuk mengukur seberapa bagus model kita terbentuk selama proses training, dana **testing data** digunakan untuk menguji performa model setelah proses training selesai dilakukan.

In [ ]:
#Acak data input sehingga terdapat distribusi untuk training, testing, dan validasi
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# masukkan ke matriks sesuai dengan indeks
inputs = inputs[randomize]
outputs = outputs[randomize]

# Pecah menjadi: training, testing & validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")

## Buat dan Latih Model

Buat dan latih [TensorFlow](https://www.tensorflow.org) model menggunakan high-level [Keras](https://www.tensorflow.org/guide/keras) API.

Model terdiri dari 2-layer tersembunyi. Layer-1 berisikan 50 neuron, sedangkan layer-2 berisi 15 neuron. Layer output terdiri dari 2 neuron, masing-masing untuk setiap gestur.

Fungsi aktivasi 'relu' digunakan pada layer tersembunyi, sedangkan 'softmax' pada layer output. Gunakan 'rmsprop' sebagai optimizer, dan 'mse' sebagai loss function. Lakukan training sebanyak 600 eppoch.

In [ ]:
# Gunakan Fully Connected Layer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu')) 
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax'))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=1, validation_data=(inputs_validate, outputs_validate))



## Lakukan Verifikasi 

Ploting performa model saat training vs saat validasi.



### Plot nilai loss

Buat grafik untuk mengetahui nilai loss

In [ ]:
# Perbesar jendela untuk ploting. Ukuran default adalah (6,4).
plt.rcParams["figure.figsize"] = (20,10)

# Tampilkan nilai loss. Gunakan "mean squared error" sebagai loss function
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(plt.rcParams["figure.figsize"])

### Zoom Lebih Dekat

Perbesar bagian akhir proses training dengan melakukan proting dari epoch ke-200 sampai dengan epoch terakhir (ke-600)

In [ ]:
# Mulai dari epoch ke-200
SKIP = 200
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Ploting Mean Absolute Error (MAE)

Performa dari model dapat diukur, salah satunya dengan menggunakan [Mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error).



In [ ]:
# Plot MAE
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Mean Absolute Error daru Proses Training & Validasi')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()


### Lakukan Prediksi terhadap Test Data
Lakukan prediksi terhadap data yang belum pernah diketahui oleh model (test data) untuk melihat seberapa bagus model kita melakukan klasifikasi terhadap data baru.


In [ ]:
# Gunakan model untuk melakukan prediksi terhadap data testing
predictions = model.predict(inputs_test)

# Tampilkan hasil dan nilai yang diharapkan (aktual)
print("predictions =\n", np.around(predictions, decimals=3))
print("actual =\n", outputs_test)

# Plot nilai prediksi dan nilai aktual
plt.clf()
plt.title('Training data predicted vs actual values')
plt.plot(inputs_test, outputs_test, 'b.', label='Actual')
plt.plot(inputs_test, predictions, 'r.', label='Predicted')
plt.show()

# Konversi Model TensorFlow ke TensorFlow Lite

Untuk mengoptimasi ukuran model, langkah selanjutnya adalah melakukan konversi dari model TensorFlow ke TensorFlo Lite (TFLite model). 

Ukuran akhir dari TFLite model ditampilkan di layar, setelah proses konversi ini selesai dijalankan.

In [ ]:
# Tanpa melakukan proses kuantisasi, ubah model menjadi TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model ke drive, kemudia hasilnya dapat diunduh
open("gesture_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)
  
  

## Konversi Model TFLite menjadi File Header Arduino

Langkah berikutnya adalah mengubah model yang dihasilkan oleh Tensorflow (TFlite model) menjadi format yang dapat dimengerti oleh mikrokontroler. Format yang dihasilkan adalah byte array, dengan format heksadesimal.

Setelah selesai menjalankan script di bawah ini, maka akan dihasilkan file dengan nama **model.h** di sisi sebelah kiri dari layar ini (File Panel), satu lokasi dengan telungkup.csv dan tengadah.csv. Double-click file tersebut dan simpanlah di komputer kita.

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

# Klasifikasi

Setelah mendapatkan model.h, maka tambahkan file model.h tersebut ke script Arduino untuk melakukan klasifikasi.
